In [ ]:
# Import required libraries
import ndjson
import pandas as pd
from google.cloud import aiplatform
from google.cloud import storage

# Initialize Vertex AI connection
aiplatform.init(location='europe-west4')

# Define Cloud Storage client
client = storage.Client()

In [ ]:
# Get the batch prediction job
job_id = '751954902724378624'
batch_prediction_job = aiplatform.BatchPredictionJob(job_id)

In [ ]:
# Get the job output folder
results_folder = batch_prediction_job.output_info.gcs_output_directory
results_folder

In [ ]:
# Create prefix to filter blobs
prefix_filter = results_folder.replace('gs://c4ds-europe-west4/','')
prefix_filter

In [ ]:
# Define Cloud Storage client and bucket to which files will be exported
client = storage.Client()
bucket = client.get_bucket('c4ds-europe-west4')

In [ ]:
# Create iterator
blobs = client.list_blobs('c4ds-europe-west4', prefix=prefix_filter)

# Show all files with predictions
for blob in blobs:
    print(blob.name)

In [ ]:
blobs = client.list_blobs('c4ds-europe-west4', prefix=prefix_filter)

# Display few predictions to show prediction format
for blob in blobs:
    # Load data and transform into json
    json_data_string = blob.download_as_string()
    json_data = ndjson.loads(json_data_string)
    pred_examples = json_data[0:3]
    break
    
pred_examples

In [ ]:
blobs = client.list_blobs('c4ds-europe-west4', prefix=prefix_filter)

# Create placeholders for prediction results
cs_uri_list = []
predicted_label_list = []
prediction_confidence_list = []

# Iterate over each file in Cloud Storage
for blob in blobs:
    # Load data and transform into json
    json_data_string = blob.download_as_string()
    json_data = ndjson.loads(json_data_string) 
    
    # Iterate over each prediction in file
    for prediction in json_data:
        # Get data from prediction
        cs_uri = prediction.get('instance').get('content')
        predicted_label = prediction.get('prediction').get('displayNames')[0] # This array is sorted so the highest probability label is always on 1st position.
        prediction_confidence = prediction.get('prediction').get('confidences')[0]
        
        # Append to appropriate list
        cs_uri_list.append(cs_uri)
        predicted_label_list.append(predicted_label)
        prediction_confidence_list.append(prediction_confidence)


In [ ]:
# Create data frame with results. 
results_df = pd.DataFrame({'cs_uri': cs_uri_list, 'predicted_label': predicted_label_list, 'confidence': prediction_confidence_list})
results_df.head()